In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import schedule
import time
import random

from sklearn import datasets, linear_model
from sklearn.linear_model import Lasso

from sklearn.externals import joblib

def mm(x,a=2,b = 0.5):
    y = a*pow(x,b)  
    return y

In [2]:
df_read = pd.read_csv('export.csv')
column_itunes = ['artist_id', 'trend', 'track_id', 'paid_youtube_impressions',
   'paid_youtube_views', 'paid_facebook_clicks',
   'paid_facebook_impressions', 'paid_twitter_impressions',
   'paid_facebook_video', 'paid_facebook_instagram_video_impressions',
   'paid_facebook_instagram_post_impressions', 'paid_twitter_clicks',
   'paid_facebook_instagram_video_clicks',
   'paid_facebook_instagram_post_clicks', 'itunes_downloads',
   'capitalfm_com_impressions','capitalfm_com_clicks', 
   'loopme_impressions', 'loopme_clicks',
   'jet_capital_impacts', 'jet_kiss_impacts', 'jet_heart_impacts']
column_spotify = ['artist_id', 'trend', 'track_id', 'paid_youtube_impressions',
   'paid_youtube_views', 'paid_facebook_clicks',
   'paid_facebook_impressions', 'paid_twitter_impressions',
   'paid_facebook_video', 'paid_facebook_instagram_video_impressions',
   'paid_facebook_instagram_post_impressions', 'paid_twitter_clicks',
   'paid_facebook_instagram_video_clicks',
   'paid_facebook_instagram_post_clicks', 'spotify_streams',
   'capitalfm_com_impressions','capitalfm_com_clicks', 
   'loopme_impressions', 'loopme_clicks',
   'jet_capital_impacts', 'jet_kiss_impacts', 'jet_heart_impacts']
col = ['paid_youtube_impressions',
   'paid_youtube_views', 'paid_facebook_clicks',
   'paid_facebook_impressions', 'paid_twitter_impressions',
   'paid_facebook_video', 'paid_facebook_instagram_video_impressions',
   'paid_facebook_instagram_post_impressions', 'paid_twitter_clicks',
   'paid_facebook_instagram_video_clicks',
   'paid_facebook_instagram_post_clicks',
   'capitalfm_com_impressions','capitalfm_com_clicks', 
   'loopme_impressions', 'loopme_clicks',
   'jet_capital_impacts', 'jet_kiss_impacts', 'jet_heart_impacts']


In [3]:
df_spotify = df_read[column_spotify]

df_spotify[col] = df_spotify[col].apply(lambda x:mm(x))
df_spotify['ma']= df_spotify.groupby('track_id')['spotify_streams'].apply(pd.rolling_mean,1, min_periods=1)
df_spotify['ma'] =df_spotify.groupby('track_id')['ma'].shift(1)
df_spotify['ma1']= df_spotify.groupby('track_id')['spotify_streams'].apply(pd.rolling_mean,2, min_periods=1)
df_spotify['ma1'] =df_spotify.groupby('track_id')['ma1'].shift(1)
df_spotify['std'] = df_spotify.groupby('track_id')['spotify_streams'].apply(pd.rolling_std,2, min_periods=1)
df_spotify['std'] = df_spotify.groupby('track_id')['std'].shift(1)
csv_spotify = df_spotify[['artist_id','track_id','trend','ma','ma1','std']]
csv_spotify = csv_spotify.fillna(0)

/home/paperspace/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/home/paperspace/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:707: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=1,min_periods=1,center=False).mean()
  return func(g, *args, **kwargs)
/home/paperspace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

In [4]:
d_spotify = pd.DataFrame(csv_spotify)
filename = 'CSV/artist_model_spotify.csv'
d_spotify.to_csv(filename, index=False, encoding='utf-8')
#df_spotify = df_spotify[df_spotify['spotify_streams'] > 10]
#df_spotify = df_spotify[df_spotify['spotify_streams'] < 4524003]
d1_spotify = df_spotify[df_spotify['spotify_streams'] > 0]
d1_spotify = d1_spotify[d1_spotify['spotify_streams'] < 4524003]
d1_spotify = d1_spotify.reset_index(drop=True)
d1_spotify = d1_spotify[pd.notnull(d1_spotify['spotify_streams'])]
df1_spotify=d1_spotify.fillna(0)

df1_spotify.drop(['track_id'],axis=1,inplace = True)
#df1_spotify.drop(['artist_id'],axis=1,inplace = True)
df1_spotify['artist_id'] = df1_spotify['artist_id'].astype('category')
#df1_spotify['track_id'] = df1_spotify['track_id'].astype('category')

In [5]:
df2_spotify = pd.get_dummies(df1_spotify, dummy_na=True)
df3_spotify=df2_spotify
df3_spotify=pd.DataFrame.sort_index(df3_spotify,axis=1)
Y_train_spotify = df3_spotify['spotify_streams']
X_train_spotify = df3_spotify.drop('spotify_streams',axis=1)



In [6]:
T_train_xgb = xgb.DMatrix(X_train_spotify, Y_train_spotify)

keys =[X_train_spotify.columns.get_loc(c) for c in X_train_spotify.columns if c in col]
feature_monotones = [0] * (len(X_train_spotify.columns) - len (keys))
for key in keys :
    feature_monotones.insert(key-1,1)
monotone_constraints = '(' + ','.join([str(m) for m in feature_monotones]) + ')'


params = {"objective": "reg:linear",
    "eta": 0.05,"max_depth" : 10,
   "subsample" : 0.8,"colsample_bytree" : 0.8,
  "eval_metric" : "rmse","nthread" : 100,"silent":1,'gamma':0, 'lambda':0.01,
  'min_child_weight':50,'monotone_constraints':monotone_constraints}

#bst_cv = xgb.cv(params, T_train_xgb, 500, nfold = 5, early_stopping_rounds=10,seed = 101)
gbm = xgb.train(params, T_train_xgb, num_boost_round = 500)

joblib.dump(gbm, 'models/artist-reg-spotify.pkl')

model_columns = X_train_spotify.columns
joblib.dump(model_columns, 'models/artist-model-spotify-col.pkl')

['models/artist-model-spotify-col.pkl']

In [ ]:
!python plan-api.py

 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Mar/2018 06:00:47] "POST /const HTTP/1.1" 200 -
